In [1]:
%load_ext autoreload
%autoreload 2
import sys
print(sys.executable)
import socket
print(socket.gethostname())
import os
os.chdir("/home/icb/alioguz.can/projects/campa_pt")

/home/icb/alioguz.can/miniconda3/envs/campa_pt/bin/python
gpusrv34.scidom.de


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
TF_ENABLE_ONEDNN_OPTS=0

In [3]:
from pathlib import Path
from campa.tl import (
    Cluster,
    Estimator,
    TorchEstimator,
    Predictor,
    TorchPredictor,
    Experiment,
    TorchExperiment,
    ModelComparator,
    run_experiments,
    run_torch_experiments
)
from campa.data import MPPData
from campa.utils import init_logging
from campa.constants import campa_config
# init logging with level INFO=20, WARNING=30
init_logging(level=30)
# read correct campa_config -- created with setup.ipynb
CAMPA_DIR = Path.cwd()
campa_config.config_fname = CAMPA_DIR / "notebooks/params/campa.ini"
print(campa_config)

/home/icb/alioguz.can/miniconda3/envs/campa_pt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-15 20:32:30.261846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-15 20:32:31.601583: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-01-15 20:32:31.601707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

Reading config from /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini
CAMPAConfig (fname: /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini)
EXPERIMENT_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments
BASE_DATA_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_data
CO_OCC_CHUNK_SIZE: 10000000.0
data_config/exampledata: /home/icb/alioguz.can/projects/campa_pt/notebooks/params/ExampleData_constants.py



In [4]:
# import tensorflow as tf
# from tensorflow.keras import backend as K
# import torch
# torch.cuda.empty_cache()
# # Clear the current session
# K.clear_session()

# # Optionally, reset the default graph
# tf.compat.v1.reset_default_graph()

In [40]:
# get Experiments from config
exps = Experiment.get_experiments_from_config("notebooks/params/example_experiment_params.py")

In [41]:
# run_experiments(exps, mode="trainval")

In [42]:
exp = exps[2]
print("Experiment name:", exp.name)
print("Experiment is stored in:", exp.full_path)

Experiment name: MPPleiden
Experiment is stored in: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/MPPleiden


In [43]:
est = Estimator(exps[2])

TypeError: 'NoneType' object is not subscriptable

In [27]:
print(Estimator(exps[0]).model.summary)

Model: "VAEModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 3, 3, 34)]   0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 3, 3, 32)     1120        ['input_12[0][0]']               
                                                                                                  
 flatten_4 (Flatten)            (None, 288)          0           ['conv2d_4[0][0]']               
                                                                                                  
 dense_14 (Dense)               (None, 32)           9248        ['flatten_4[0][0]']              
                                                                                           

In [12]:
_ = est.train_model()

4/4 [==============================] - 10s 346ms/step - loss: 5223.8276 - decoder_loss: 5223.8062 - latent_loss: 0.0221 - decoder_mean_squared_error: 0.7407 - latent_kl_loss: 0.0223 - val_loss: 5243.8989 - val_decoder_loss: 5243.8716 - val_latent_loss: 0.0274 - val_decoder_mean_squared_error: 0.7189 - val_latent_kl_loss: 0.0274


In [21]:
pred_torch = Predictor(exps[1])
pred_torch.evaluate_model()

319/319 [==============================] - 1s 3ms/step


In [22]:
print(est.model.model.summary())

Model: "VAEModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14)]         0           []                               
                                                                                                  
 condition_encoder (Functional)  (None, 10)          260         ['input_3[0][0]']                
                                                                                                  
 input_1 (InputLayer)           [(None, 3, 3, 34)]   0           []                               
                                                                                                  
 lambda (Lambda)                (None, 3, 3, 10)     0           ['condition_encoder[0][0]']      
                                                                                           

In [30]:
torch_exps = TorchExperiment.get_experiments_from_config("notebooks/params/example_experiment_params_torch.py")
print("Experiment name:", torch_exps[2].name)
print("Experiment is stored in:", torch_exps[1].full_path)

Experiment name: MPPleiden
Experiment is stored in: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/CondVAE_pert-CC


In [45]:
est_torch = TorchEstimator(torch_exps[0])
print(est_torch.model)
est_torch.model.total_trainable_params()

VAEModelTorch(
  (encoder): Sequential(
    (0): Conv2d(34, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=288, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=16, bias=True)
    (6): ReLU()
  )
  (latent): Linear(in_features=16, out_features=32, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=34, bias=True)
  )
)
12018


In [46]:
est_torch.train_model()

Epochs: 100%|██████████| 1/1 [00:00<00:00, 35.97it/s]


{'epoch': 0,
 'loss': 4481.19189453125,
 'val_loss': 4582.275390625,
 'decoder_loss': 4481.179870605469,
 'latent_loss': 0.01203770306892693}

In [47]:
pred_torch = TorchPredictor(torch_exps[0])
pred_torch.evaluate_model()

In [48]:
results_folder = os.path.join(pred_torch.exp.full_path, f"results_epoch{pred_torch.est.epoch:03d}")
print("Results folder", results_folder)
print(os.listdir(results_folder))

Results folder /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch000
['val', 'val_imgs']


In [49]:
print(MPPData.from_data_dir(os.path.join(results_folder, "val"), data_config="ExampleData"))

MPPData for ExampleData (123 mpps with shape (3, 3, 34) from 8 objects). Data keys: ['obj_ids', 'y', 'x', 'mpp', 'labels', 'conditions', 'latent'].


In [50]:
cl = Cluster.from_exp_split(torch_exps[0])

Cannot read with memmap:  /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch001/val/clustering.npy


AssertionError: 

In [11]:
print(cl.config["leiden_resolution"])
cl.create_clustering()

0.2


/ictstr01/home/icb/alioguz.can/projects/campa_pt/campa/tl/_cluster.py:598: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(


In [44]:
run_torch_experiments(torch_exps, mode="trainval")

Running experiment for ['VAE', 'CondVAE_pert-CC', 'MPPleiden'] with mode trainval
Training model for VAE


Epochs: 100%|██████████| 1/1 [00:00<00:00,  2.00it/s]


Evaluating model for VAE
Clustering results for VAE
Cannot read with memmap:  /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch001/val/clustering.npy


AssertionError: 